# Replicating Pesaran & Timmermann (1994) for UK Equity Markets
## Complete Implementation Guide for Forecasting FTSE All-Share Excess Returns

**Author:** Manpreet Sangha [210048348]  
**Course:** SMM265 Asset Pricing  
**Date:** December 2025

### Executive Summary

This notebook replicates the methodology from:
> Pesaran, M. H. and Timmermann, A. (1994) "Forecasting Stock Returns: An Examination of Stock Market Trading in the Presence of Transaction Costs", Journal of Forecasting, Vol. 13, pp. 335-367.

**Adaptation for UK Markets:**
- **Market:** FTSE All-Share Index
- **Forecast Horizon:** 6 months (semi-annual)  
- **Evaluation Period:** October 2015 – October 2025
- **Training Data:** 1990 – October 2015 (expanding window)

**Key Objectives:**
1. Predict excess stock returns using macroeconomic variables
2. Implement recursive out-of-sample forecasting
3. Evaluate directional accuracy using Pesaran-Timmermann sign test
4. Test switching strategy performance vs buy-and-hold
5. Analyze robustness under transaction costs

**Expected Deliverables:**
- Regression model with UK predictors (DY, PI12, DI12, DIP12, TERM)
- 20 recursive out-of-sample predictions (6-month periods)
- Statistical significance tests of forecasting ability
- Portfolio performance analysis with transaction cost scenarios

## 1. Import Required Libraries and Setup

In [ ]:
# Import required libraries for the analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import os
from pathlib import Path

# Statistical and econometric libraries
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan, acorr_breusch_godfrey
from statsmodels.stats.stattools import jarque_bera, durbin_watson
from statsmodels.stats.outliers_influence import reset_ramsey

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Configure plotting parameters
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Set random seed for reproducibility
np.random.seed(42)

# Define global constants
BASE_PATH = Path(r"C:\Users\Manpreet\OneDrive - City St George's, University of London\Documents\Term1\Coursework\SMM265 Asset Pricing\Q1")
RAW_DATA_PATH = BASE_PATH / "Data" / "Raw Data"
CLEANED_DATA_PATH = BASE_PATH / "Data" / "Cleaned Data"
OUTPUT_PATH = BASE_PATH / "Python Scripts" / "Asset-Pricing" / "outputs"

# Create output directories if they don't exist
for path in [OUTPUT_PATH, OUTPUT_PATH / "tables", OUTPUT_PATH / "figures"]:
    path.mkdir(parents=True, exist_ok=True)

# Key dates for analysis
TRAINING_START = '1990-01-31'  # Start of training period
FORECAST_START = '2015-10-31'  # Start of out-of-sample period
FORECAST_END = '2025-10-31'    # End of evaluation period
FORECAST_HORIZON = 6           # 6-month forecast horizon

print("Libraries imported successfully!")
print(f"Analysis period: {TRAINING_START} to {FORECAST_END}")
print(f"Out-of-sample evaluation: {FORECAST_START} to {FORECAST_END}")
print(f"Forecast horizon: {FORECAST_HORIZON} months")
print(f"Output directory: {OUTPUT_PATH}")

# Display key thresholds from the paper
print("\n" + "="*60)
print("EXPECTED RESULTS BASED ON PESARAN & TIMMERMANN (1994)")
print("="*60)
print("Expected R²: 0.15 - 0.30 (between quarterly and annual)")
print("Expected Sign Accuracy: 55% - 70%")
print("Expected PT Statistic: 1.5 - 2.5")
print("Critical Values - PT Test: 1.645 (5%), 2.326 (1%)")
print("="*60)

## 2. Load and Process Financial Data

This section loads all required financial and economic data:
- **FTSE All-Share Total Return Index** (monthly, 1990-2025)
- **FTSE All-Share Dividend Yield** (monthly, 1990-2025)
- **UK 3-Month Treasury Bill Rate** (monthly, 1989-2025)
- **UK Consumer Price Index (CPI)** (monthly, 1989-2025)
- **UK Industrial Production Index** (monthly, 1989-2025)
- **UK 10-Year Gilt Yield** (optional, monthly, 1990-2025)

**Data Sources:**
- Bank of England Statistical Database
- Office for National Statistics (ONS)
- Yahoo Finance / Bloomberg for equity data

In [ ]:
def load_excel_data(file_pattern, data_type="financial", monthly_path="Monthly", 
                   skip_rows=5, date_col=0, value_col=1, 
                   date_name="Date", value_name="Value"):
    """
    Generic function to load and clean Excel data files.
    
    Parameters:
    -----------
    file_pattern : str
        Pattern to match Excel files
    data_type : str
        Description for logging
    monthly_path : str
        Subfolder containing monthly data
    skip_rows : int
        Number of rows to skip from top
    date_col : int
        Column index for dates
    value_col : int
        Column index for values
    date_name : str
        Name for date column
    value_name : str
        Name for value column
        
    Returns:
    --------
    pd.DataFrame or None
    """
    try:
        # Look for files matching the pattern
        data_dir = CLEANED_DATA_PATH / monthly_path
        files = list(data_dir.glob(file_pattern))
        
        if not files:
            print(f"⚠️  No files found matching pattern: {file_pattern}")
            print(f"   Looking in: {data_dir}")
            return None
            
        file_path = files[0]  # Use first match
        print(f"📁 Loading {data_type}: {file_path.name}")
        
        # Load data
        df = pd.read_excel(file_path)
        
        # Skip metadata rows
        if skip_rows > 0:
            df = df.iloc[skip_rows:].copy()
        
        # Clean empty rows/columns
        df = df.dropna(how='all').dropna(axis=1, how='all').reset_index(drop=True)
        
        # Select and rename columns
        if len(df.columns) >= max(date_col + 1, value_col + 1):
            df = df.iloc[:, [date_col, value_col]].copy()
            df.columns = [date_name, value_name]
        else:
            print(f"⚠️  Expected at least {max(date_col + 1, value_col + 1)} columns, found {len(df.columns)}")
            return None
        
        # Convert date column
        df[date_name] = pd.to_datetime(df[date_name], errors='coerce')
        
        # Convert value column to numeric
        df[value_name] = pd.to_numeric(df[value_name], errors='coerce')
        
        # Remove rows with invalid dates or values
        df = df.dropna().reset_index(drop=True)
        
        # Sort by date
        df = df.sort_values(date_name).reset_index(drop=True)
        
        print(f"   ✅ Loaded {len(df)} observations from {df[date_name].min().strftime('%Y-%m')} to {df[date_name].max().strftime('%Y-%m')}")
        
        return df
        
    except Exception as e:
        print(f"❌ Error loading {data_type}: {e}")
        return None


def create_sample_data():
    """
    Create sample data for demonstration if real data files are not available.
    This generates realistic synthetic data based on historical UK patterns.
    """
    print("📊 Creating sample data for demonstration...")
    
    # Create date range from 1989 to 2025 (monthly)
    dates = pd.date_range(start='1989-01-31', end='2025-10-31', freq='M')
    n_obs = len(dates)
    
    # Set random seed for reproducibility
    np.random.seed(42)
    
    # Create synthetic data based on realistic UK historical patterns
    
    # FTSE All-Share (starts at 1000, trending upward with volatility)
    ftse_growth = 0.005  # 0.5% average monthly growth
    ftse_vol = 0.04      # 4% monthly volatility
    ftse_returns = np.random.normal(ftse_growth, ftse_vol, n_obs)
    ftse_prices = [1000]  # Starting value
    for ret in ftse_returns:
        ftse_prices.append(ftse_prices[-1] * (1 + ret))
    ftse_prices = ftse_prices[1:]  # Remove initial value
    
    # Dividend Yield (starts around 4%, mean-reverting)
    div_yield_mean = 3.5
    div_yield_vol = 0.5
    div_yield = div_yield_mean + np.random.normal(0, div_yield_vol, n_obs).cumsum() * 0.1
    div_yield = np.clip(div_yield, 1.0, 7.0)  # Keep between 1% and 7%
    
    # UK 3-Month T-Bill Rate (starts around 10%, trending down)
    tbill_trend = np.linspace(10, 1, n_obs)  # Declining trend from 10% to 1%
    tbill_vol = 0.3
    tbill_rate = tbill_trend + np.random.normal(0, tbill_vol, n_obs).cumsum() * 0.1
    tbill_rate = np.clip(tbill_rate, 0.1, 15.0)  # Keep positive and reasonable
    
    # UK CPI (starting at 100, inflation trend)
    cpi_growth = 0.002  # 0.2% average monthly inflation
    cpi_vol = 0.003     # 0.3% monthly volatility
    cpi_returns = np.random.normal(cpi_growth, cpi_vol, n_obs)
    cpi_values = [100]  # Starting at 100
    for ret in cpi_returns:
        cpi_values.append(cpi_values[-1] * (1 + ret))
    cpi_values = cpi_values[1:]
    
    # UK Industrial Production (starting at 100, cyclical)
    ip_trend = 0.001    # 0.1% average monthly growth
    ip_vol = 0.01       # 1% monthly volatility
    ip_cycle = 0.5 * np.sin(2 * np.pi * np.arange(n_obs) / 60)  # 5-year cycle
    ip_returns = np.random.normal(ip_trend, ip_vol, n_obs) + ip_cycle * 0.01
    ip_values = [100]   # Starting at 100
    for ret in ip_returns:
        ip_values.append(ip_values[-1] * (1 + ret))
    ip_values = ip_values[1:]
    
    # UK 10-Year Gilt (related to T-bill with spread)
    gilt_spread = 1.5 + np.random.normal(0, 0.5, n_obs).cumsum() * 0.1
    gilt_spread = np.clip(gilt_spread, 0, 4)  # Keep spread positive and reasonable
    gilt_yield = tbill_rate + gilt_spread
    
    # Create DataFrames
    datasets = {
        'ftse': pd.DataFrame({'Date': dates, 'ftse_price': ftse_prices}),
        'dividend': pd.DataFrame({'Date': dates, 'dividend_yield': div_yield}),
        'tbill': pd.DataFrame({'Date': dates, 'uktb_yield': tbill_rate}),
        'cpi': pd.DataFrame({'Date': dates, 'cpi_index': cpi_values}),
        'ip': pd.DataFrame({'Date': dates, 'ip_index': ip_values}),
        'gilt': pd.DataFrame({'Date': dates, 'gilt_yield': gilt_yield})
    }
    
    print(f"   ✅ Created sample data with {n_obs} monthly observations")
    print(f"   📅 Period: {dates[0].strftime('%Y-%m')} to {dates[-1].strftime('%Y-%m')}")
    
    return datasets


# Load all required datasets
print("🚀 LOADING FINANCIAL AND ECONOMIC DATA")
print("="*50)

# Try to load real data first, fall back to sample data if unavailable
data = {}

# Load each dataset
file_patterns = {
    'ftse': '*FTSE*All*Share*.xlsx',
    'dividend': '*dividend*yield*.xlsx',
    'tbill': '*Treasury*Bill*.xlsx',
    'cpi': '*CPI*.xlsx', 
    'ip': '*Industrial*Production*.xlsx',
    'gilt': '*Gilt*.xlsx'
}

value_names = {
    'ftse': 'ftse_price',
    'dividend': 'dividend_yield', 
    'tbill': 'uktb_yield',
    'cpi': 'cpi_index',
    'ip': 'ip_index',
    'gilt': 'gilt_yield'
}

# Try loading real data
real_data_loaded = 0
for key, pattern in file_patterns.items():
    df = load_excel_data(pattern, data_type=key.upper(), value_name=value_names[key])
    if df is not None:
        data[key] = df
        real_data_loaded += 1

# If most real data is missing, use sample data
if real_data_loaded < 3:
    print("\n⚠️  Limited real data available. Using sample data for demonstration.")
    print("   For actual coursework, replace with real data files.\n")
    data = create_sample_data()
else:
    print(f"\n✅ Successfully loaded {real_data_loaded} real datasets")

print(f"\n📊 Available datasets: {list(data.keys())}")
print("="*50)

## 3. Construct Predictor Variables

Following Pesaran & Timmermann (1994), we construct five key predictor variables:

| Variable | Formula | Lag | Expected Sign | Economic Rationale |
|----------|---------|-----|---------------|-------------------|
| **DY** | Dividend yield (level) | t−1 | **+** | High yield = undervalued = higher future returns |
| **PI12** | log(CPI_t / CPI_{t-12}) | t−2 | **−** | High inflation = uncertainty = lower returns |
| **DI12** | I3m_t − I3m_{t-12} | t−1 | **−** | Rising rates = tighter money = lower returns |
| **DIP12** | log(IP_t / IP_{t-12}) | t−2 | **−** | Strong economy = low risk premium = lower expected returns |
| **TERM** | I10y_t − I3m_t | t−1 | **+** | Wide spread = recession expected = higher risk premium |

**Key Points:**
- All variables use appropriate lags to avoid look-ahead bias
- 2-month lag for macro data (CPI, IP) reflects publication delays
- 1-month lag for financial data (rates, yields) reflects immediate availability

In [ ]:
def construct_predictor_variables(data_dict):
    """
    Construct all predictor variables following Pesaran & Timmermann (1994).
    
    Parameters:
    -----------
    data_dict : dict
        Dictionary containing loaded datasets
    
    Returns:
    --------
    pd.DataFrame
        Combined dataset with all predictor variables
    """
    
    print("🔧 CONSTRUCTING PREDICTOR VARIABLES")
    print("="*50)
    
    # Start with the most complete dataset (usually FTSE)
    base_df = data_dict['ftse'].copy()
    base_df = base_df.set_index('Date')
    
    print(f"📅 Base period: {base_df.index.min().strftime('%Y-%m')} to {base_df.index.max().strftime('%Y-%m')}")
    
    # Merge all datasets on date
    for key, df in data_dict.items():
        if key != 'ftse':
            df_merge = df.set_index('Date')
            base_df = base_df.join(df_merge, how='outer')
    
    print(f"📊 Combined dataset shape: {base_df.shape}")
    print(f"📊 Available columns: {list(base_df.columns)}")
    
    # Forward fill missing values (up to 2 periods for monthly data)
    base_df = base_df.fillna(method='ffill', limit=2)
    
    # Sort by date to ensure proper time series order
    base_df = base_df.sort_index()
    
    # Construct predictor variables
    print("\n🏗️  Constructing individual predictors...")
    
    # 1. Dividend Yield (DY) - level, lagged 1 month
    if 'dividend_yield' in base_df.columns:
        base_df['DY'] = base_df['dividend_yield'].shift(1) / 100  # Convert to decimal and lag
        print("   ✅ DY: Dividend yield (t-1)")
    else:
        print("   ⚠️  Dividend yield data not available - using proxy")
        # Use a proxy based on FTSE level (inverse relationship)
        ftse_ma = base_df['ftse_price'].rolling(12).mean()
        base_df['DY'] = (4.0 * ftse_ma.iloc[0] / ftse_ma).shift(1) / 100  # Starts at 4%
    
    # 2. 12-month Inflation (PI12) - log difference, lagged 2 months
    if 'cpi_index' in base_df.columns:
        base_df['PI12'] = (np.log(base_df['cpi_index']) - np.log(base_df['cpi_index'].shift(12))).shift(2)
        print("   ✅ PI12: 12-month inflation (t-2)")
    else:
        print("   ⚠️  CPI data not available - using synthetic inflation")
        base_df['PI12'] = (np.random.normal(0.02, 0.01, len(base_df))).shift(2)
    
    # 3. 12-month Change in Interest Rate (DI12) - difference, lagged 1 month
    if 'uktb_yield' in base_df.columns:
        base_df['DI12'] = (base_df['uktb_yield'] - base_df['uktb_yield'].shift(12)).shift(1)
        print("   ✅ DI12: 12-month rate change (t-1)")
    else:
        print("   ⚠️  T-bill data not available - using synthetic rates")
        synthetic_rate = 5 + np.random.normal(0, 2, len(base_df)).cumsum() * 0.1
        base_df['uktb_yield'] = synthetic_rate
        base_df['DI12'] = (synthetic_rate - pd.Series(synthetic_rate).shift(12)).shift(1)
    
    # 4. 12-month Change in Industrial Production (DIP12) - log difference, lagged 2 months  
    if 'ip_index' in base_df.columns:
        base_df['DIP12'] = (np.log(base_df['ip_index']) - np.log(base_df['ip_index'].shift(12))).shift(2)
        print("   ✅ DIP12: 12-month IP change (t-2)")
    else:
        print("   ⚠️  IP data not available - using synthetic IP")
        base_df['DIP12'] = (np.random.normal(0.01, 0.02, len(base_df))).shift(2)
    
    # 5. Term Spread (TERM) - long-short rate difference, lagged 1 month
    if 'gilt_yield' in base_df.columns and 'uktb_yield' in base_df.columns:
        base_df['TERM'] = (base_df['gilt_yield'] - base_df['uktb_yield']).shift(1)
        print("   ✅ TERM: Term spread (t-1)")
    else:
        print("   ⚠️  Gilt data not available - creating synthetic term spread")
        if 'uktb_yield' in base_df.columns:
            base_df['TERM'] = (base_df['uktb_yield'] + 1.5 + np.random.normal(0, 0.5, len(base_df))).shift(1)
        else:
            base_df['TERM'] = np.random.normal(1.5, 0.5, len(base_df))
    
    print(f"\n📋 Summary of constructed variables:")
    predictor_cols = ['DY', 'PI12', 'DI12', 'DIP12', 'TERM']
    for col in predictor_cols:
        if col in base_df.columns:
            valid_obs = base_df[col].notna().sum()
            mean_val = base_df[col].mean()
            std_val = base_df[col].std()
            print(f"   {col:6}: {valid_obs:4d} obs, mean={mean_val:7.4f}, std={std_val:7.4f}")
    
    return base_df


def create_excess_returns(df, horizon=6):
    """
    Calculate 6-month excess returns for FTSE All-Share.
    
    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame with FTSE prices and T-bill rates
    horizon : int
        Forecast horizon in months
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with excess returns added
    """
    
    print(f"\n📈 CALCULATING {horizon}-MONTH EXCESS RETURNS")
    print("="*50)
    
    df = df.copy()
    
    # Calculate FTSE total returns (assuming prices include dividends)
    # 6-month nominal return: (Price_t / Price_{t-6}) - 1
    df['ftse_return_6m'] = (df['ftse_price'] / df['ftse_price'].shift(horizon)) - 1
    
    # Calculate 6-month risk-free return
    # Convert annual T-bill rate to 6-month rate
    df['rf_6m'] = ((1 + df['uktb_yield'] / 100) ** (horizon/12)) - 1
    
    # Shift risk-free rate to align with return period
    # Use rate known at beginning of 6-month period
    df['rf_6m_lagged'] = df['rf_6m'].shift(horizon)
    
    # Calculate excess returns
    df['excess_return_6m'] = df['ftse_return_6m'] - df['rf_6m_lagged']
    
    # Calculate basic statistics
    valid_excess = df['excess_return_6m'].dropna()
    
    print(f"📊 Excess return statistics:")
    print(f"   Observations: {len(valid_excess)}")
    print(f"   Mean: {valid_excess.mean():.4f} ({valid_excess.mean()*100:.2f}%)")
    print(f"   Std Dev: {valid_excess.std():.4f} ({valid_excess.std()*100:.2f}%)")
    print(f"   Min: {valid_excess.min():.4f} ({valid_excess.min()*100:.2f}%)")
    print(f"   Max: {valid_excess.max():.4f} ({valid_excess.max()*100:.2f}%)")
    print(f"   Positive periods: {(valid_excess > 0).sum()} ({(valid_excess > 0).mean()*100:.1f}%)")
    
    return df


# Execute variable construction
model_data = construct_predictor_variables(data)
model_data = create_excess_returns(model_data, horizon=FORECAST_HORIZON)

# Display final dataset info
print(f"\n📋 FINAL MODEL DATASET")
print("="*50)
print(f"Shape: {model_data.shape}")
print(f"Period: {model_data.index.min().strftime('%Y-%m')} to {model_data.index.max().strftime('%Y-%m')}")
print(f"Columns: {list(model_data.columns)}")

# Check for missing values in key variables
key_vars = ['excess_return_6m', 'DY', 'PI12', 'DI12', 'DIP12', 'TERM']
missing_summary = model_data[key_vars].isnull().sum()
print(f"\nMissing values by variable:")
for var in key_vars:
    if var in model_data.columns:
        missing = missing_summary[var] if var in missing_summary else 0
        print(f"   {var:15}: {missing:4d} missing")

print("\n✅ Variable construction completed!")

## 4. Calculate Excess Returns

The dependent variable is the **6-month excess return** on FTSE All-Share:

**ERFTSE_t = NRFTSE_t − RF_{t-6}**

Where:
- **NRFTSE_t** = 6-month nominal return on FTSE All-Share (including dividends)
- **RF_{t-6}** = 6-month risk-free rate known at start of period

**Key Implementation Details:**
- Use Total Return Index when available (includes dividend reinvestment)
- Convert annual T-bill rate to 6-month equivalent: `(1 + annual_rate)^0.5 - 1`
- Align timing: risk-free rate known at beginning of forecast period
- Handle missing data through forward-filling and interpolation

## 5. Full Sample Regression Analysis

Estimate the baseline regression model using all available data:

**Primary Model (4 predictors):**
```
ERFTSE_t = α + β₁×DY_{t-1} + β₂×PI12_{t-2} + β₃×DI12_{t-1} + β₄×DIP12_{t-2} + ε_t
```

**Extended Model (5 predictors):**
```
ERFTSE_t = α + β₁×DY_{t-1} + β₂×PI12_{t-2} + β₃×DI12_{t-1} + β₄×DIP12_{t-2} + β₅×TERM_{t-1} + ε_t
```

**Expected Coefficient Signs:**
- DY (Dividend Yield): **Positive** (+) - High yield indicates undervaluation
- PI12 (Inflation): **Negative** (−) - High inflation reduces real returns  
- DI12 (Rate Change): **Negative** (−) - Rising rates hurt equity valuations
- DIP12 (IP Change): **Negative** (−) - Strong economy reduces risk premium
- TERM (Term Spread): **Positive** (+) - Wide spread indicates recession risk

**Diagnostic Tests:**
- Durbin-Watson (serial correlation)
- Jarque-Bera (normality)
- Breusch-Pagan (heteroscedasticity)
- RESET (functional form)

In [ ]:
def run_full_sample_regression(df, predictors=['DY', 'PI12', 'DI12', 'DIP12'], 
                              target='excess_return_6m', include_term=False):
    """
    Run full sample OLS regression with diagnostic tests.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Model dataset
    predictors : list
        List of predictor variable names
    target : str
        Target variable name
    include_term : bool
        Whether to include term spread
        
    Returns:
    --------
    dict
        Regression results and diagnostics
    """
    
    print("📊 FULL SAMPLE REGRESSION ANALYSIS")
    print("="*60)
    
    # Prepare data
    if include_term and 'TERM' not in predictors:
        predictors = predictors + ['TERM']
    
    # Create regression dataset (drop missing values)
    reg_vars = [target] + predictors
    reg_data = df[reg_vars].dropna()
    
    print(f"📋 Regression specification:")
    print(f"   Dependent variable: {target}")
    print(f"   Predictors: {', '.join(predictors)}")
    print(f"   Sample size: {len(reg_data)} observations")
    print(f"   Sample period: {reg_data.index.min().strftime('%Y-%m')} to {reg_data.index.max().strftime('%Y-%m')}")
    
    # Prepare variables
    y = reg_data[target]
    X = reg_data[predictors]
    X_with_const = sm.add_constant(X)  # Add intercept
    
    # Estimate OLS regression
    model = sm.OLS(y, X_with_const).fit()
    
    print(f"\n📈 REGRESSION RESULTS")
    print("="*60)
    print(model.summary())
    
    # Extract key statistics
    results = {
        'model': model,
        'n_obs': len(reg_data),
        'r_squared': model.rsquared,
        'adj_r_squared': model.rsquared_adj,
        'f_statistic': model.fvalue,
        'f_pvalue': model.f_pvalue,
        'coefficients': dict(zip(['const'] + predictors, model.params)),
        'std_errors': dict(zip(['const'] + predictors, model.bse)),
        't_stats': dict(zip(['const'] + predictors, model.tvalues)),
        'p_values': dict(zip(['const'] + predictors, model.pvalues)),
        'residuals': model.resid,
        'fitted_values': model.fittedvalues
    }
    
    # Diagnostic tests
    print(f"\n🔍 DIAGNOSTIC TESTS")
    print("="*60)
    
    # 1. Durbin-Watson test (serial correlation)
    dw_stat = durbin_watson(results['residuals'])
    print(f"Durbin-Watson statistic: {dw_stat:.3f}")
    if dw_stat < 1.5:
        print("   → Positive serial correlation detected")
    elif dw_stat > 2.5:
        print("   → Negative serial correlation detected") 
    else:
        print("   → No strong evidence of serial correlation")
    
    # 2. Jarque-Bera test (normality)
    jb_stat, jb_pvalue = jarque_bera(results['residuals'])
    print(f"Jarque-Bera test: {jb_stat:.3f} (p-value: {jb_pvalue:.4f})")
    if jb_pvalue < 0.05:
        print("   → Residuals are not normally distributed (expected for returns)")
    else:
        print("   → Cannot reject normality")
    
    # 3. Breusch-Pagan test (heteroscedasticity)
    bp_stat, bp_pvalue, _, _ = het_breuschpagan(results['residuals'], X_with_const)
    print(f"Breusch-Pagan test: {bp_stat:.3f} (p-value: {bp_pvalue:.4f})")
    if bp_pvalue < 0.05:
        print("   → Heteroscedasticity detected")
    else:
        print("   → Homoscedasticity cannot be rejected")
    
    # 4. RESET test (functional form)
    try:
        reset_stat, reset_pvalue, _ = reset_ramsey(results['residuals'], X_with_const)
        print(f"RESET test: {reset_stat:.3f} (p-value: {reset_pvalue:.4f})")
        if reset_pvalue < 0.05:
            print("   → Model specification may be inadequate")
        else:
            print("   → No evidence of specification error")
    except:
        print("RESET test: Could not compute (insufficient observations)")
        reset_stat, reset_pvalue = np.nan, np.nan
    
    # Store diagnostic results
    results.update({
        'durbin_watson': dw_stat,
        'jarque_bera': (jb_stat, jb_pvalue),
        'breusch_pagan': (bp_stat, bp_pvalue),
        'reset': (reset_stat, reset_pvalue)
    })
    
    # Coefficient analysis
    print(f"\n📋 COEFFICIENT ANALYSIS")
    print("="*60)
    expected_signs = {'DY': '+', 'PI12': '-', 'DI12': '-', 'DIP12': '-', 'TERM': '+'}
    
    print(f"{'Variable':<8} {'Coeff':<10} {'Std Err':<10} {'t-stat':<8} {'p-value':<8} {'Expected':<8} {'Sign OK?':<8}")
    print("-" * 70)
    
    for var in ['const'] + predictors:
        if var in results['coefficients']:
            coef = results['coefficients'][var]
            se = results['std_errors'][var] 
            t_stat = results['t_stats'][var]
            p_val = results['p_values'][var]
            
            if var == 'const':
                expected = 'N/A'
                sign_ok = 'N/A'
            else:
                expected = expected_signs.get(var, '?')
                actual_sign = '+' if coef > 0 else '-'
                sign_ok = '✓' if actual_sign == expected else '✗'
            
            print(f"{var:<8} {coef:>9.4f} {se:>9.4f} {t_stat:>7.2f} {p_val:>7.4f} {expected:>7s} {sign_ok:>7s}")
    
    return results


def create_regression_table(results, title="Regression Results", save_path=None):
    """
    Create a formatted regression results table.
    """
    
    # Extract information
    n_obs = results['n_obs']
    r2 = results['r_squared']
    adj_r2 = results['adj_r_squared'] 
    f_stat = results['f_statistic']
    f_pval = results['f_pvalue']
    dw = results['durbin_watson']
    
    # Build table
    table_lines = []
    table_lines.append(f"{title}")
    table_lines.append("=" * 70)
    table_lines.append(f"{'Variable':<15} {'Coefficient':<12} {'Std Error':<12} {'t-statistic':<12} {'p-value':<10}")
    table_lines.append("-" * 70)
    
    # Add coefficients
    vars_order = ['const', 'DY', 'PI12', 'DI12', 'DIP12', 'TERM']
    for var in vars_order:
        if var in results['coefficients']:
            coef = results['coefficients'][var]
            se = results['std_errors'][var]
            t_stat = results['t_stats'][var] 
            p_val = results['p_values'][var]
            
            var_label = 'Constant' if var == 'const' else var
            table_lines.append(f"{var_label:<15} {coef:>11.4f} {se:>11.4f} {t_stat:>11.2f} {p_val:>9.4f}")
    
    table_lines.append("-" * 70)
    table_lines.append(f"{'Observations':<15} {n_obs:>11d}")
    table_lines.append(f"{'R²':<15} {r2:>11.4f}")
    table_lines.append(f"{'Adjusted R²':<15} {adj_r2:>11.4f}")
    table_lines.append(f"{'F-statistic':<15} {f_stat:>11.2f}")
    table_lines.append(f"{'Prob(F-stat)':<15} {f_pval:>11.4f}")
    table_lines.append(f"{'Durbin-Watson':<15} {dw:>11.3f}")
    table_lines.append("=" * 70)
    
    table_text = "\n".join(table_lines)
    print(table_text)
    
    if save_path:
        with open(save_path, 'w') as f:
            f.write(table_text)
    
    return table_text


# Run full sample regressions
print("🚀 RUNNING FULL SAMPLE ANALYSIS")
print("="*70)

# Prepare clean dataset for regression (remove missing values)
clean_data = model_data.dropna(subset=['excess_return_6m', 'DY', 'PI12', 'DI12', 'DIP12'])

print(f"📊 Clean regression dataset:")
print(f"   Original observations: {len(model_data)}")
print(f"   Clean observations: {len(clean_data)}")
print(f"   Data coverage: {clean_data.index.min().strftime('%Y-%m')} to {clean_data.index.max().strftime('%Y-%m')}")

# Run primary regression (4 predictors)
print("\n" + "="*70)
print("ESTIMATING PRIMARY MODEL (4 PREDICTORS)")
print("="*70)
primary_results = run_full_sample_regression(
    clean_data, 
    predictors=['DY', 'PI12', 'DI12', 'DIP12'],
    include_term=False
)

# Create and save regression table
table_primary = create_regression_table(
    primary_results, 
    title="Table 1: OLS Regression Results - Primary Model (4 Predictors)",
    save_path=OUTPUT_PATH / "tables" / "regression_primary.txt"
)

# Run extended regression (5 predictors) if term spread available
if 'TERM' in clean_data.columns and clean_data['TERM'].notna().sum() > 50:
    print("\n" + "="*70) 
    print("ESTIMATING EXTENDED MODEL (5 PREDICTORS)")
    print("="*70)
    
    extended_results = run_full_sample_regression(
        clean_data,
        predictors=['DY', 'PI12', 'DI12', 'DIP12'],
        include_term=True
    )
    
    table_extended = create_regression_table(
        extended_results,
        title="Table 2: OLS Regression Results - Extended Model (5 Predictors)", 
        save_path=OUTPUT_PATH / "tables" / "regression_extended.txt"
    )
else:
    print("\n⚠️  Term spread data insufficient for extended model")
    extended_results = None

print(f"\n✅ Full sample regression analysis completed!")
print(f"📁 Results saved to: {OUTPUT_PATH / 'tables'}")

## 6. Implement Recursive Forecasting Procedure

The core of the Pesaran-Timmermann methodology is **recursive out-of-sample forecasting** using an expanding window:

**Procedure for each decision date t:**
1. **COLLECT** training data from Jan 1990 to t
2. **ESTIMATE** regression: `ERFTSE = α + β₁×DY + β₂×PI12 + β₃×DI12 + β₄×DIP12 + ε`
3. **OBTAIN** predictor values at time t (with appropriate lags)
4. **PREDICT** excess return for next 6 months: `ρ̂_{t+6} = α̂ + β̂₁×DY_{t-1} + β̂₂×PI12_{t-2} + β̂₃×DI12_{t-1} + β̂₄×DIP12_{t-2}`
5. **DECIDE** trading position: Stocks if ρ̂_{t+6} > 0, Bonds otherwise
6. **RECORD** prediction, actual outcome, and trading decision
7. **EXPAND** training window and repeat

**Timeline:**
- **Training starts:** January 1990
- **First prediction:** November 2015 - April 2016  
- **Last prediction:** May 2025 - October 2025
- **Total predictions:** 20 periods (6-month intervals)

**No Look-Ahead Bias:** Each prediction uses only information available at the decision date.

In [ ]:
def generate_forecast_dates(start_date, end_date, horizon=6):
    """
    Generate decision dates for recursive forecasting.
    
    Parameters:
    -----------
    start_date : str
        Start of out-of-sample period
    end_date : str  
        End of evaluation period
    horizon : int
        Forecast horizon in months
        
    Returns:
    --------
    list of tuples
        (decision_date, forecast_start, forecast_end)
    """
    
    decision_dates = []
    current_date = pd.to_datetime(start_date)
    end_dt = pd.to_datetime(end_date)
    
    while current_date <= end_dt:
        # Calculate forecast period 
        forecast_start = current_date + pd.DateOffset(months=1)
        forecast_end = forecast_start + pd.DateOffset(months=horizon-1)
        
        # Only include if forecast_end is within our data range
        if forecast_end <= end_dt:
            decision_dates.append((current_date, forecast_start, forecast_end))
        
        # Move to next decision date (6 months later)
        current_date += pd.DateOffset(months=horizon)
    
    return decision_dates


def run_recursive_forecasting(df, predictors=['DY', 'PI12', 'DI12', 'DIP12'],
                            target='excess_return_6m', 
                            training_start='1990-01-31',
                            forecast_start='2015-10-31',
                            forecast_end='2025-10-31',
                            horizon=6):
    """
    Execute recursive out-of-sample forecasting procedure.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Model dataset with all variables
    predictors : list
        List of predictor variables  
    target : str
        Target variable name
    training_start : str
        Start of training period
    forecast_start : str
        Start of out-of-sample forecasting
    forecast_end : str
        End of evaluation period
    horizon : int
        Forecast horizon in months
        
    Returns:
    --------
    pd.DataFrame
        Results with predictions and actuals
    """
    
    print("🔮 RECURSIVE OUT-OF-SAMPLE FORECASTING")
    print("="*70)
    
    # Generate forecast dates
    forecast_dates = generate_forecast_dates(forecast_start, forecast_end, horizon)
    
    print(f"📅 Forecast schedule:")
    print(f"   Training start: {training_start}")
    print(f"   First forecast: {forecast_dates[0][1].strftime('%Y-%m')} to {forecast_dates[0][2].strftime('%Y-%m')}")  
    print(f"   Last forecast: {forecast_dates[-1][1].strftime('%Y-%m')} to {forecast_dates[-1][2].strftime('%Y-%m')}")
    print(f"   Total forecasts: {len(forecast_dates)}\")\n")
    
    results = []\n    training_start_dt = pd.to_datetime(training_start)
    
    for i, (decision_date, forecast_start_date, forecast_end_date) in enumerate(forecast_dates):
        
        print(f"🎯 Forecast {i+1:2d}/{len(forecast_dates)}: Decision {decision_date.strftime('%Y-%m-%d')} → Forecast {forecast_start_date.strftime('%Y-%m')} to {forecast_end_date.strftime('%Y-%m')}")
        
        # 1. Collect training data up to decision date
        training_mask = (df.index >= training_start_dt) & (df.index <= decision_date)
        training_data = df[training_mask].copy()
        
        # Remove rows with missing values in regression variables
        reg_vars = [target] + predictors
        training_clean = training_data[reg_vars].dropna()
        
        if len(training_clean) < 20:  # Minimum sample size
            print(f"   ⚠️  Insufficient training data: {len(training_clean)} obs")
            continue
        
        # 2. Estimate regression model
        y_train = training_clean[target]
        X_train = training_clean[predictors] 
        X_train_const = sm.add_constant(X_train)
        
        try:
            model = sm.OLS(y_train, X_train_const).fit()
        except:
            print(f"   ❌ Regression estimation failed")
            continue
        
        # 3. Get predictor values at decision date (with appropriate lags)
        # Predictors are already lagged in construction, so use decision_date values
        try:
            predictor_values = df.loc[decision_date, predictors]
            
            # Check for missing predictor values
            if predictor_values.isna().any():
                print(f"   ⚠️  Missing predictor values at decision date")
                print(f"      Missing: {predictor_values[predictor_values.isna()].index.tolist()}")
                # Forward fill from recent values
                for var in predictors:
                    if pd.isna(predictor_values[var]):
                        recent_values = df.loc[:decision_date, var].dropna()
                        if len(recent_values) > 0:
                            predictor_values[var] = recent_values.iloc[-1]
                            print(f"      {var}: filled with {predictor_values[var]:.4f}")
        
        except KeyError:
            print(f"   ❌ Decision date {decision_date} not in dataset")
            continue
        
        # 4. Make prediction
        X_forecast = np.concatenate([[1], predictor_values.values])  # Add constant
        predicted_excess_return = np.dot(model.params, X_forecast)
        
        # 5. Get actual outcome (if available)
        actual_excess_return = np.nan
        actual_stock_return = np.nan
        actual_bond_return = np.nan
        
        # Find actual return over forecast period
        forecast_dates_in_data = df.index[(df.index >= forecast_start_date) & (df.index <= forecast_end_date)]
        if len(forecast_dates_in_data) > 0:
            # Use the last available observation in the forecast period
            actual_date = forecast_dates_in_data[-1]
            try:
                actual_excess_return = df.loc[actual_date, target]
                # Calculate components for trading strategy
                actual_stock_return = df.loc[actual_date, 'ftse_return_6m'] 
                actual_bond_return = df.loc[actual_date, 'rf_6m']
            except KeyError:
                pass
        
        # 6. Make trading decision
        trading_decision = 'STOCKS' if predicted_excess_return > 0 else 'BONDS'
        
        # 7. Record results
        result = {
            'decision_date': decision_date,
            'forecast_start': forecast_start_date,
            'forecast_end': forecast_end_date,
            'training_observations': len(training_clean),
            'predicted_excess_return': predicted_excess_return,
            'actual_excess_return': actual_excess_return,
            'actual_stock_return': actual_stock_return,
            'actual_bond_return': actual_bond_return,
            'trading_decision': trading_decision,
            'model_r_squared': model.rsquared,
            'predictor_values': dict(predictor_values)
        }
        
        results.append(result)
        
        # Display prediction
        print(f"      Training obs: {len(training_clean):3d}, R² = {model.rsquared:.3f}")
        print(f"      Predicted ER: {predicted_excess_return:7.4f} → {trading_decision}")
        if not pd.isna(actual_excess_return):
            sign_correct = (predicted_excess_return > 0) == (actual_excess_return > 0)
            print(f"      Actual ER:    {actual_excess_return:7.4f} → Sign correct: {sign_correct}")
        print()
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    
    print(f"✅ Recursive forecasting completed!")
    print(f"   Generated {len(results_df)} predictions")
    
    # Calculate summary statistics
    if len(results_df) > 0:
        valid_actuals = results_df['actual_excess_return'].dropna()
        valid_predictions = results_df.loc[valid_actuals.index, 'predicted_excess_return']
        
        if len(valid_actuals) > 0:
            # Sign accuracy
            sign_accuracy = ((valid_predictions > 0) == (valid_actuals > 0)).mean()
            
            print(f"   Sign accuracy: {sign_accuracy:.1%} ({(valid_predictions > 0).sum()}/{len(valid_predictions)})")
            print(f"   Mean prediction: {valid_predictions.mean():.4f}")
            print(f"   Mean actual: {valid_actuals.mean():.4f}")
            print(f"   Correlation: {valid_predictions.corr(valid_actuals):.3f}")
    
    return results_df


# Execute recursive forecasting
print("🚀 STARTING RECURSIVE FORECASTING")
print("="*70)

# Run recursive forecasting with primary model
recursive_results = run_recursive_forecasting(
    df=clean_data,
    predictors=['DY', 'PI12', 'DI12', 'DIP12'],
    target='excess_return_6m',
    training_start=TRAINING_START,
    forecast_start=FORECAST_START, 
    forecast_end=FORECAST_END,
    horizon=FORECAST_HORIZON
)

# Display results summary
print(f"\n📊 RECURSIVE FORECASTING SUMMARY")
print("="*70)
print(f"Total predictions generated: {len(recursive_results)}")
print(f"Period: {recursive_results['forecast_start'].min().strftime('%Y-%m')} to {recursive_results['forecast_end'].max().strftime('%Y-%m')}")

# Save results
recursive_results.to_csv(OUTPUT_PATH / "recursive_forecasting_results.csv", index=False)
recursive_results.to_excel(OUTPUT_PATH / "recursive_forecasting_results.xlsx", index=False)

print(f"📁 Results saved to: {OUTPUT_PATH}")
print("✅ Recursive forecasting completed successfully!")

## 7. Execute Pesaran-Timmermann Sign Test

The **Pesaran-Timmermann (PT) test** evaluates whether the model predicts the **direction** of excess returns better than random chance. 

**Key Features:**
- **Distribution-free** (works with non-normal returns)
- **Focuses on direction** (what matters for trading)  
- **Statistically rigorous** (formal hypothesis test)

**Test Procedure:**
1. Count correct sign predictions: `P̂ = n_correct / n`
2. Calculate expected accuracy under independence: `P* = P_y × P_ŷ + (1-P_y) × (1-P_ŷ)`
3. Compute test statistic: `PT = (P̂ - P*) / SE`
4. Compare to critical values: 1.645 (5%), 2.326 (1%)

**Economic Interpretation:**
- **H₀:** Signs are independent (no forecasting ability)
- **H₁:** Model has directional forecasting ability  
- **Rejection:** Evidence of genuine forecasting skill

In [ ]:
def pesaran_timmermann_test(actual, predicted):
    """
    Compute the Pesaran-Timmermann directional accuracy test.
    
    Parameters:
    -----------
    actual : array-like
        Actual excess returns
    predicted : array-like
        Predicted excess returns
        
    Returns:
    --------
    dict
        Test results including statistic, p-value, and accuracy metrics
    """
    
    actual = np.array(actual)
    predicted = np.array(predicted)
    n = len(actual)
    
    if n == 0:
        return {'error': 'No valid observations'}
    
    # Convert to signs (1 = positive, 0 = negative/zero)
    sign_actual = (actual > 0).astype(int)
    sign_predicted = (predicted > 0).astype(int)
    
    # Proportion of correct sign predictions
    correct = (sign_actual == sign_predicted)
    P_hat = correct.mean()
    
    # Proportions of positive returns
    P_y = sign_actual.mean()        # Proportion of positive actual returns
    P_y_hat = sign_predicted.mean() # Proportion of positive predicted returns
    
    # Expected proportion correct under null hypothesis (independence)
    P_star = P_y * P_y_hat + (1 - P_y) * (1 - P_y_hat)
    
    # Variance calculations
    # Variance of P_hat under null
    Var_P_hat = P_star * (1 - P_star) / n
    
    # Variance of P_star  
    Var_P_star = ((2 * P_y - 1)**2 * P_y_hat * (1 - P_y_hat) / n +
                  (2 * P_y_hat - 1)**2 * P_y * (1 - P_y) / n +
                  4 * P_y * P_y_hat * (1 - P_y) * (1 - P_y_hat) / n**2)
    
    # Standard error
    SE = np.sqrt(Var_P_hat - Var_P_star) if (Var_P_hat - Var_P_star) > 0 else 0
    
    # Test statistic  
    if SE > 0:
        PT_statistic = (P_hat - P_star) / SE
        p_value = 1 - stats.norm.cdf(PT_statistic)  # One-sided test
    else:
        PT_statistic = np.nan
        p_value = np.nan
    
    return {
        'n_predictions': n,
        'n_correct': int(correct.sum()),
        'accuracy': P_hat,
        'expected_accuracy': P_star,
        'prop_positive_actual': P_y,
        'prop_positive_predicted': P_y_hat,
        'PT_statistic': PT_statistic,
        'p_value': p_value,
        'standard_error': SE,
        'variance_P_hat': Var_P_hat,
        'variance_P_star': Var_P_star,
        'significant_5pct': PT_statistic >= 1.645 if not np.isnan(PT_statistic) else False,
        'significant_1pct': PT_statistic >= 2.326 if not np.isnan(PT_statistic) else False
    }


def create_sign_test_table(pt_results, title="Pesaran-Timmermann Sign Test Results"):
    """
    Create formatted table for PT test results.
    """
    
    table_lines = []
    table_lines.append(title)
    table_lines.append("=" * 65)
    table_lines.append(f"{'Metric':<35} {'Value':<20} {'Notes':<10}")
    table_lines.append("-" * 65)
    
    # Basic statistics
    table_lines.append(f"{'Number of predictions':<35} {pt_results['n_predictions']:<20d}")
    table_lines.append(f"{'Correct sign predictions':<35} {pt_results['n_correct']:<20d}")
    table_lines.append(f"{'Proportion correct (%)':<35} {pt_results['accuracy']*100:<20.1f}")
    table_lines.append(f"{'Expected proportion under H0 (%)':<35} {pt_results['expected_accuracy']*100:<20.1f}")
    
    table_lines.append("-" * 65)
    
    # Test statistics
    table_lines.append(f"{'PT test statistic':<35} {pt_results['PT_statistic']:<20.3f}")
    table_lines.append(f"{'p-value (one-sided)':<35} {pt_results['p_value']:<20.4f}")
    table_lines.append(f"{'Standard error':<35} {pt_results['standard_error']:<20.4f}")
    
    table_lines.append("-" * 65)
    
    # Significance
    sig_5 = "Yes" if pt_results['significant_5pct'] else "No"
    sig_1 = "Yes" if pt_results['significant_1pct'] else "No" 
    table_lines.append(f"{'Significant at 5%?':<35} {sig_5:<20}")
    table_lines.append(f"{'Significant at 1%?':<35} {sig_1:<20}")
    
    table_lines.append("-" * 65)
    
    # Additional metrics
    table_lines.append(f"{'Prop. positive actual (%)':<35} {pt_results['prop_positive_actual']*100:<20.1f}")
    table_lines.append(f"{'Prop. positive predicted (%)':<35} {pt_results['prop_positive_predicted']*100:<20.1f}")
    
    table_lines.append("=" * 65)
    
    # Add interpretation
    table_lines.append("")
    table_lines.append("INTERPRETATION:")
    if pt_results['significant_5pct']:
        table_lines.append(f"✅ Model demonstrates significant directional forecasting ability")
        table_lines.append(f"   (PT statistic {pt_results['PT_statistic']:.3f} > 1.645 critical value)")
    else:
        table_lines.append(f"❌ No significant evidence of directional forecasting ability") 
        table_lines.append(f"   (PT statistic {pt_results['PT_statistic']:.3f} < 1.645 critical value)")
    
    if pt_results['accuracy'] > 0.6:
        table_lines.append(f"📈 High sign accuracy: {pt_results['accuracy']*100:.1f}% correct predictions")
    elif pt_results['accuracy'] > 0.55:
        table_lines.append(f"📊 Moderate sign accuracy: {pt_results['accuracy']*100:.1f}% correct predictions")
    else:
        table_lines.append(f"📉 Low sign accuracy: {pt_results['accuracy']*100:.1f}% correct predictions")
    
    table_text = "\n".join(table_lines)
    return table_text


# Execute Pesaran-Timmermann test
print("🧪 PESARAN-TIMMERMANN SIGN TEST")
print("="*70)

# Extract predictions and actuals (only valid pairs)
valid_mask = recursive_results['actual_excess_return'].notna()
valid_results = recursive_results[valid_mask].copy()

if len(valid_results) == 0:
    print("❌ No valid prediction-actual pairs available for testing")
else:
    actual_returns = valid_results['actual_excess_return'].values
    predicted_returns = valid_results['predicted_excess_return'].values
    
    print(f"📊 Test data summary:")
    print(f"   Valid predictions: {len(actual_returns)}")
    print(f"   Actual returns - positive: {(actual_returns > 0).sum()}/{len(actual_returns)} ({(actual_returns > 0).mean()*100:.1f}%)")
    print(f"   Predicted returns - positive: {(predicted_returns > 0).sum()}/{len(predicted_returns)} ({(predicted_returns > 0).mean()*100:.1f}%)")
    
    # Run PT test
    pt_results = pesaran_timmermann_test(actual_returns, predicted_returns)
    
    if 'error' in pt_results:
        print(f"❌ Test failed: {pt_results['error']}")
    else:
        print(f"\n📋 SIGN TEST RESULTS")
        print("-" * 70)
        
        # Create and display results table
        sign_test_table = create_sign_test_table(pt_results)
        print(sign_test_table)
        
        # Save results
        with open(OUTPUT_PATH / "tables" / "pesaran_timmermann_test.txt", 'w') as f:
            f.write(sign_test_table)
        
        # Additional analysis
        print(f"\n📈 DETAILED SIGN ANALYSIS")
        print("-" * 70)
        
        # Create contingency table
        sign_actual = (actual_returns > 0).astype(int)
        sign_predicted = (predicted_returns > 0).astype(int)
        
        # 2x2 contingency table
        correct_pos = ((sign_actual == 1) & (sign_predicted == 1)).sum()  # True positives
        correct_neg = ((sign_actual == 0) & (sign_predicted == 0)).sum()  # True negatives  
        wrong_pos = ((sign_actual == 0) & (sign_predicted == 1)).sum()    # False positives
        wrong_neg = ((sign_actual == 1) & (sign_predicted == 0)).sum()    # False negatives
        
        print(f"Contingency Table:")
        print(f"                    Predicted")
        print(f"Actual         Negative  Positive  Total")
        print(f"Negative       {correct_neg:8d}  {wrong_pos:8d}  {correct_neg + wrong_pos:5d}")
        print(f"Positive       {wrong_neg:8d}  {correct_pos:8d}  {wrong_neg + correct_pos:5d}")
        print(f"Total          {correct_neg + wrong_neg:8d}  {wrong_pos + correct_pos:8d}  {len(actual_returns):5d}")
        
        print(f"\nPrediction Quality:")
        if len(actual_returns) > 0:
            sensitivity = correct_pos / (correct_pos + wrong_neg) if (correct_pos + wrong_neg) > 0 else 0
            specificity = correct_neg / (correct_neg + wrong_pos) if (correct_neg + wrong_pos) > 0 else 0
            print(f"   Sensitivity (correct positive): {sensitivity:.1%}")
            print(f"   Specificity (correct negative): {specificity:.1%}")
        
        print(f"\n✅ Pesaran-Timmermann test completed!")
        print(f"📁 Results saved to: {OUTPUT_PATH / 'tables' / 'pesaran_timmermann_test.txt'}")

## 8. Implement Trading Strategy with Transaction Costs

Implement the **switching strategy** based on model predictions:

**Trading Rule:**
- **IF** predicted_excess_return > 0: **HOLD STOCKS** (FTSE All-Share)
- **ELSE**: **HOLD BONDS** (UK T-bills)

**Transaction Cost Scenarios:**
- **Zero (0.0%):** Benchmark for pure model performance  
- **Low (0.25%):** Modern ETF/index fund trading
- **Medium (0.5%):** Realistic for individual investors
- **High (1.0%):** Conservative estimate/historical costs

**Cost Structure:**
- Applied only when **switching positions** (Stocks ↔ Bonds)
- No costs for **holding** the same position
- Bonds assumed to be held to maturity (no transaction costs)

**Performance Metrics:**
- Mean 6-month returns
- Standard deviation (risk measure) 
- Sharpe ratio (risk-adjusted performance)
- Total cumulative return
- Maximum drawdown

In [ ]:
def calculate_switching_strategy_returns(results_df, transaction_costs=[0.0, 0.0025, 0.005, 0.01]):
    """
    Calculate returns for switching strategy under different transaction cost scenarios.
    
    Parameters:
    -----------
    results_df : pd.DataFrame
        Recursive forecasting results
    transaction_costs : list
        List of transaction cost rates to test
        
    Returns:
    --------
    dict
        Performance results by transaction cost scenario
    """
    
    print("💼 TRADING STRATEGY IMPLEMENTATION")
    print("="*70)
    
    # Filter to valid observations only
    valid_data = results_df[
        results_df['actual_excess_return'].notna() & 
        results_df['actual_stock_return'].notna() & 
        results_df['actual_bond_return'].notna()
    ].copy()
    
    if len(valid_data) == 0:
        print("❌ No valid trading periods available")
        return {}
    
    print(f"📊 Trading analysis:")
    print(f"   Valid trading periods: {len(valid_data)}")
    print(f"   Period: {valid_data['forecast_start'].min().strftime('%Y-%m')} to {valid_data['forecast_end'].max().strftime('%Y-%m')}")
    
    # Extract data
    predictions = valid_data['predicted_excess_return'].values
    stock_returns = valid_data['actual_stock_return'].values
    bond_returns = valid_data['actual_bond_return'].values
    trading_decisions = valid_data['trading_decision'].values
    
    # Count trading decisions
    stocks_periods = (trading_decisions == 'STOCKS').sum()
    bonds_periods = (trading_decisions == 'BONDS').sum()
    
    print(f"   Stock positions: {stocks_periods} ({stocks_periods/len(valid_data)*100:.1f}%)")
    print(f"   Bond positions: {bonds_periods} ({bonds_periods/len(valid_data)*100:.1f}%)")
    
    # Calculate number of switches
    switches = 0
    for i in range(1, len(trading_decisions)):
        if trading_decisions[i] != trading_decisions[i-1]:
            switches += 1
    
    print(f"   Total switches: {switches}")
    
    # Performance analysis for each transaction cost scenario
    cost_names = ['Zero', 'Low (0.25%)', 'Medium (0.5%)', 'High (1.0%)']
    performance_results = {}
    
    print(f"\n📈 STRATEGY PERFORMANCE BY TRANSACTION COST")
    print("="*70)
    
    for i, cost in enumerate(transaction_costs):
        cost_name = cost_names[i] if i < len(cost_names) else f"{cost*100:.2f}%"
        
        # Calculate portfolio returns
        portfolio_returns = []
        previous_position = None
        
        for j in range(len(predictions)):
            # Determine position based on prediction
            current_position = 'STOCKS' if predictions[j] > 0 else 'BONDS'
            
            # Get gross return
            if current_position == 'STOCKS':
                gross_return = stock_returns[j]
            else:
                gross_return = bond_returns[j]
            
            # Apply transaction cost if switching
            if previous_position is not None and current_position != previous_position:
                net_return = gross_return - cost
            else:
                net_return = gross_return
            
            portfolio_returns.append(net_return)
            previous_position = current_position
        
        portfolio_returns = np.array(portfolio_returns)
        
        # Calculate performance metrics
        mean_return = np.mean(portfolio_returns)
        std_return = np.std(portfolio_returns, ddof=1) if len(portfolio_returns) > 1 else 0
        total_return = np.prod(1 + portfolio_returns) - 1
        
        # Annualized metrics (6-month periods → multiply by 2 for annual)
        annual_mean = mean_return * 2
        annual_std = std_return * np.sqrt(2)
        sharpe_ratio = annual_mean / annual_std if annual_std > 0 else 0
        
        # Maximum drawdown
        cumulative_returns = np.cumprod(1 + portfolio_returns)
        running_max = np.maximum.accumulate(cumulative_returns)
        drawdown = (cumulative_returns - running_max) / running_max
        max_drawdown = np.min(drawdown)
        
        # Store results
        performance_results[cost_name] = {
            'transaction_cost': cost,
            'mean_return': mean_return,
            'std_return': std_return,
            'annual_mean': annual_mean,
            'annual_std': annual_std,
            'sharpe_ratio': sharpe_ratio,
            'total_return': total_return,
            'max_drawdown': max_drawdown,
            'portfolio_returns': portfolio_returns,
            'n_switches': switches
        }
        
        # Display results
        print(f"\n{cost_name} Transaction Costs ({cost*100:.2f}%):")
        print(f"   Mean 6-month return: {mean_return*100:6.2f}% (Annual: {annual_mean*100:6.2f}%)")
        print(f"   Volatility:          {std_return*100:6.2f}% (Annual: {annual_std*100:6.2f}%)")
        print(f"   Sharpe ratio:        {sharpe_ratio:6.3f}")
        print(f"   Total return:        {total_return*100:6.2f}%")
        print(f"   Max drawdown:        {max_drawdown*100:6.2f}%")
    
    return performance_results


def calculate_benchmark_returns(results_df):
    """
    Calculate benchmark strategy returns for comparison.
    """
    
    print(f"\n📊 BENCHMARK STRATEGIES")
    print("="*70)
    
    # Filter valid data
    valid_data = results_df[
        results_df['actual_stock_return'].notna() & 
        results_df['actual_bond_return'].notna()
    ].copy()
    
    if len(valid_data) == 0:
        return {}
    
    stock_returns = valid_data['actual_stock_return'].values
    bond_returns = valid_data['actual_bond_return'].values
    
    benchmarks = {}
    
    # 1. Buy and Hold Stocks (FTSE All-Share)
    bh_mean = np.mean(stock_returns)
    bh_std = np.std(stock_returns, ddof=1) if len(stock_returns) > 1 else 0
    bh_total = np.prod(1 + stock_returns) - 1
    bh_annual_mean = bh_mean * 2
    bh_annual_std = bh_std * np.sqrt(2)
    bh_sharpe = bh_annual_mean / bh_annual_std if bh_annual_std > 0 else 0
    
    # Max drawdown for buy-and-hold
    bh_cumulative = np.cumprod(1 + stock_returns)
    bh_running_max = np.maximum.accumulate(bh_cumulative)
    bh_drawdown = (bh_cumulative - bh_running_max) / bh_running_max
    bh_max_drawdown = np.min(bh_drawdown)
    
    benchmarks['Buy-and-Hold FTSE'] = {
        'mean_return': bh_mean,
        'std_return': bh_std,
        'annual_mean': bh_annual_mean,
        'annual_std': bh_annual_std,
        'sharpe_ratio': bh_sharpe,
        'total_return': bh_total,
        'max_drawdown': bh_max_drawdown,
        'returns': stock_returns
    }
    
    # 2. Always Bonds (UK T-Bills)
    bonds_mean = np.mean(bond_returns)
    bonds_std = np.std(bond_returns, ddof=1) if len(bond_returns) > 1 else 0
    bonds_total = np.prod(1 + bond_returns) - 1
    bonds_annual_mean = bonds_mean * 2
    bonds_annual_std = bonds_std * np.sqrt(2)
    bonds_sharpe = bonds_annual_mean / bonds_annual_std if bonds_annual_std > 0 else 0
    
    benchmarks['Always Bonds'] = {
        'mean_return': bonds_mean,
        'std_return': bonds_std,
        'annual_mean': bonds_annual_mean,
        'annual_std': bonds_annual_std,
        'sharpe_ratio': bonds_sharpe,
        'total_return': bonds_total,
        'max_drawdown': 0.0,  # Bonds assumed risk-free
        'returns': bond_returns
    }
    
    # 3. Random Switching (50/50 each period)
    np.random.seed(42)  # For reproducibility
    random_decisions = np.random.choice(['STOCKS', 'BONDS'], size=len(stock_returns), p=[0.5, 0.5])
    random_returns = np.where(random_decisions == 'STOCKS', stock_returns, bond_returns)
    
    random_mean = np.mean(random_returns)
    random_std = np.std(random_returns, ddof=1) if len(random_returns) > 1 else 0
    random_total = np.prod(1 + random_returns) - 1
    random_annual_mean = random_mean * 2
    random_annual_std = random_std * np.sqrt(2)
    random_sharpe = random_annual_mean / random_annual_std if random_annual_std > 0 else 0
    
    benchmarks['Random Switching'] = {
        'mean_return': random_mean,
        'std_return': random_std,
        'annual_mean': random_annual_mean, 
        'annual_std': random_annual_std,
        'sharpe_ratio': random_sharpe,
        'total_return': random_total,
        'max_drawdown': 0.0,  # Not calculated for random
        'returns': random_returns
    }
    
    # Display benchmark results
    for name, metrics in benchmarks.items():
        print(f"\n{name}:")
        print(f"   Mean 6-month return: {metrics['mean_return']*100:6.2f}% (Annual: {metrics['annual_mean']*100:6.2f}%)")
        print(f"   Volatility:          {metrics['std_return']*100:6.2f}% (Annual: {metrics['annual_std']*100:6.2f}%)")
        print(f"   Sharpe ratio:        {metrics['sharpe_ratio']:6.3f}")
        print(f"   Total return:        {metrics['total_return']*100:6.2f}%")
        if metrics['max_drawdown'] < 0:
            print(f"   Max drawdown:        {metrics['max_drawdown']*100:6.2f}%")
    
    return benchmarks


# Execute trading strategy analysis
trading_performance = calculate_switching_strategy_returns(recursive_results)
benchmark_performance = calculate_benchmark_returns(recursive_results)

# Combine results for comparison
print(f"\n📋 PERFORMANCE COMPARISON SUMMARY")
print("="*70)

all_strategies = {**trading_performance, **benchmark_performance}

if all_strategies:
    # Create comparison table
    print(f"{'Strategy':<20} {'Mean Ret':<10} {'Volatility':<10} {'Sharpe':<8} {'Total Ret':<10}")
    print("-" * 70)
    
    for name, metrics in all_strategies.items():
        mean_ret = metrics['annual_mean'] * 100
        vol = metrics['annual_std'] * 100
        sharpe = metrics['sharpe_ratio']
        total_ret = metrics['total_return'] * 100
        
        print(f"{name:<20} {mean_ret:>8.2f}% {vol:>8.2f}% {sharpe:>7.3f} {total_ret:>8.2f}%")
    
    print("="*70)
    
    # Key insights
    if 'Zero' in trading_performance and 'Buy-and-Hold FTSE' in benchmark_performance:
        switching_sharpe = trading_performance['Zero']['sharpe_ratio']
        bh_sharpe = benchmark_performance['Buy-and-Hold FTSE']['sharpe_ratio']
        
        print(f"\n💡 KEY INSIGHTS:")
        if switching_sharpe > bh_sharpe:
            print(f"   ✅ Switching strategy outperforms buy-and-hold (Sharpe: {switching_sharpe:.3f} vs {bh_sharpe:.3f})")
        else:
            print(f"   ❌ Switching strategy underperforms buy-and-hold (Sharpe: {switching_sharpe:.3f} vs {bh_sharpe:.3f})")
        
        # Check transaction cost impact
        if 'High (1.0%)' in trading_performance:
            high_cost_sharpe = trading_performance['High (1.0%)']['sharpe_ratio'] 
            if high_cost_sharpe > bh_sharpe:
                print(f"   💪 Strategy robust to high transaction costs")
            else:
                print(f"   ⚠️  High transaction costs erode performance")

print(f"\n✅ Trading strategy analysis completed!")
print(f"📁 Results can be exported to: {OUTPUT_PATH}")

## 9. Performance Evaluation and Benchmarking

Compare the switching strategy against multiple benchmarks to evaluate economic significance:

**Comparison Benchmarks:**
1. **Buy-and-Hold FTSE:** Always hold stocks (market performance)
2. **Always Bonds:** Always hold T-bills (risk-free performance) 
3. **Random Switching:** Random 50/50 allocation each period
4. **Switching Strategy:** Model-based predictions (various transaction costs)

**Performance Metrics:**
- **Mean Return:** Average period performance
- **Volatility:** Standard deviation of returns (risk measure)
- **Sharpe Ratio:** Risk-adjusted performance `(Mean - RF) / Volatility`
- **Total Return:** Cumulative wealth growth over evaluation period
- **Maximum Drawdown:** Largest peak-to-trough decline

**Success Criteria:**
- Switching strategy should outperform buy-and-hold on risk-adjusted basis
- Performance should be robust to realistic transaction costs  
- Sharpe ratio improvement demonstrates genuine economic value

## 10. Generate Summary Tables and Visualizations

Create comprehensive tables and charts for coursework submission: